In [2]:
from cnfgen import *
import cnfgen
from itertools import product
import plotly.express as px
from itertools import combinations
from math import factorial
from rich.progress import track
import pandas as pd
from typing import *
import multiprocessing as mp
from functools import partial
import networkx as nx
import random
import json
def stats(F,print_res: bool = False):
    vars = list(F.variables())
    clauses = list(F.clauses())
    n_vars = len(vars)
    n_clauses = len(clauses)
    if print_res:
        print(f"n_vars: {n_vars}, n_clauses: {n_clauses}")
    return f"{n_vars},{n_clauses}"

## counting 

In [2]:
F = CountingPrinciple(19, 2)
stats(F,print_res=True)
F = CountingPrinciple(17, 2)
stats(F,print_res=True)
F = CountingPrinciple(20, 2)
stats(F,print_res=True)
F = CountingPrinciple(20, 3)
stats(F,print_res=True)

n_vars: 171, n_clauses: 2926
n_vars: 136, n_clauses: 2057
n_vars: 190, n_clauses: 3440
n_vars: 1140, n_clauses: 290720


**Explosion du nombre de clauses mais pas des variables. Impossible d'obtenir**

## PerfectMatchingPrinciple

In [3]:
G = nx.gnm_random_graph(8000, 5000)
F = PerfectMatchingPrinciple(G)
stats(F,print_res=True)
print('-'*20)
G = nx.gnm_random_graph(1000, 1000)
F = PerfectMatchingPrinciple(G)
stats(F,print_res=True)
G = nx.gnm_random_graph(1000,2000)
F = PerfectMatchingPrinciple(G)
stats(F,print_res=True)
G = nx.gnm_random_graph(1000,3000)
F = PerfectMatchingPrinciple(G)
stats(F,print_res=True)
print('-'*20)
G = nx.gnm_random_graph(2000,3000)
F = PerfectMatchingPrinciple(G)
stats(F,print_res=True)
G = nx.gnm_random_graph(3000,3000)
F = PerfectMatchingPrinciple(G)
stats(F,print_res=True)
G = nx.gnm_random_graph(7000,3000)
F = PerfectMatchingPrinciple(G)
stats(F,print_res=True)
G = nx.gnm_random_graph(10000,3000)
F = PerfectMatchingPrinciple(G)
stats(F,print_res=True)

n_vars: 5000, n_clauses: 14285
--------------------
n_vars: 1000, n_clauses: 2988
n_vars: 2000, n_clauses: 9040
n_vars: 3000, n_clauses: 18864
--------------------
n_vars: 3000, n_clauses: 11062
n_vars: 3000, n_clauses: 8942
n_vars: 3000, n_clauses: 9500
n_vars: 3000, n_clauses: 11772


n_vars=n_edges, n_clauses = x times n_vars (bigger). n_clauses décroit quand n_nodes augmente
processus de génération:
- prendre m edges dans [1000, 5000]
- tirage aléatoire dans [1000, 10_000] de n nodes

**Potentiellement impossible d'avoir 3000 vars et moins de 5000 clauses**

## EvenColoringFormula

In [19]:
Lnodes = [10,20,30,50]
Ldeg = [10,20,30,50]
Lres = [[] for _ in range(len(Lnodes))]
for i,n_nodes in enumerate(Lnodes):
    for deg in Ldeg:
        deg = [random.randint(0, 10)*2 for _ in range(n_nodes)]
        G = nx.configuration_model(deg_sequence=deg)
        F = EvenColoringFormula(G)
        res = str(stats(F,print_res=False))
        Lres[i].append(res)
    print('-',end='')
print(pd.DataFrame(Lres, index=Lnodes, columns=Ldeg))



--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
            10          20          30          50
10      29,371       15,90      20,228       17,93
20    84,14441     67,9264     69,2918     59,3134
30   125,31300    101,8612   108,21785   127,29124
50  179,101289  209,156031  214,214320  201,142676


**Inutilisable**

## GraphColoringFormula

In [82]:
Lnodes = list(range(50,551,100))
Ledge = list(range(50,551,100))
Lcolors = list(range(4,7))
Lres = [[] for _ in range(len(Lnodes))]
for i,n_nodes in enumerate(Lnodes):
    for edge in Ledge:
        for colors in Lcolors:
            G = nx.gnm_random_graph(n_nodes, edge, seed=42)
            F = GraphColoringFormula(G,colors)
            res = str(stats(F,print_res=False))
            Lres[i].append(res)
    print('-',end='')
print()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display(pd.DataFrame(Lres, index=Lnodes, columns=[str(e)+','+str(c) for e in Ledge for c in Lcolors]))



------


,"50,4","50,5","50,6","150,4","150,5","150,6","250,4","250,5","250,6","350,4","350,5","350,6","450,4","450,5","450,6","550,4","550,5","550,6"
50,"200,550","250,800","300,1100","200,950","250,1300","300,1700","200,1350","250,1800","300,2300","200,1750","250,2300","300,2900","200,2150","250,2800","300,3500","200,2550","250,3300","300,4100"
150,"600,1250","750,1900","900,2700","600,1650","750,2400","900,3300","600,2050","750,2900","900,3900","600,2450","750,3400","900,4500","600,2850","750,3900","900,5100","600,3250","750,4400","900,5700"
250,"1000,1950","1250,3000","1500,4300","1000,2350","1250,3500","1500,4900","1000,2750","1250,4000","1500,5500","1000,3150","1250,4500","1500,6100","1000,3550","1250,5000","1500,6700","1000,3950","1250,5500","1500,7300"
350,"1400,2650","1750,4100","2100,5900","1400,3050","1750,4600","2100,6500","1400,3450","1750,5100","2100,7100","1400,3850","1750,5600","2100,7700","1400,4250","1750,6100","2100,8300","1400,4650","1750,6600","2100,8900"
450,"1800,3350","2250,5200","2700,7500","1800,3750","2250,5700","2700,8100","1800,4150","2250,6200","2700,8700","1800,4550","2250,6700","2700,9300","1800,4950","2250,7200","2700,9900","1800,5350","2250,7700","2700,10500"
550,"2200,4050","2750,6300","3300,9100","2200,4450","2750,6800","3300,9700","2200,4850","2750,7300","3300,10300","2200,5250","2750,7800","3300,10900","2200,5650","2750,8300","3300,11500","2200,6050","2750,8800","3300,12100"


**Utilisable** mais clauses très similaires d'un graphe à l'autre

## GraphAutomorphism

In [43]:
Lnodes = list(range(10,50,10))
Ledge = list(range(10,50,10))
Lres = [[] for _ in range(len(Lnodes))]
for i,n_nodes in enumerate(Lnodes):
    for edge in Ledge:
        G = nx.gnm_random_graph(n_nodes, edge)
        F = GraphAutomorphism(G)
        res = str(stats(F,print_res=False))
        Lres[i].append(res)
    print('-',end='')
print()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display(pd.DataFrame(Lres, index=Lnodes, columns=Ledge))



----


,10,20,30,40
10,"100,2321","100,2921","100,2721","100,1721"
20,"400,14841","400,21241","400,26841","400,31641"
30,"900,43161","900,59361","900,74761","900,89361"
40,"1600,93281","1600,123281","1600,152481","1600,180881"


**Inutilisable: impossible d'avoir à la fois n_var et n_clauses dans [1000,5000]**

## GraphIsomorphism

In [45]:
Lnodes = list(range(10,50,10))
Ledge = list(range(10,50,10))
Lres = [[] for _ in range(len(Lnodes))]
for i,n_nodes in enumerate(Lnodes):
    for edge in Ledge:
        G1 = nx.gnm_random_graph(n_nodes, edge,seed=random.randint(0,1000))
        G2 = nx.gnm_random_graph(n_nodes, edge,seed=random.randint(0,1000))
        F = GraphIsomorphism(G1,G2)
        res = str(stats(F,print_res=False))
        Lres[i].append(res)
    print('-',end='')
print()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display(pd.DataFrame(Lres, index=Lnodes, columns=Ledge))



----


,10,20,30,40
10,"100,2320","100,2920","100,2720","100,1720"
20,"400,14840","400,21240","400,26840","400,31640"
30,"900,43160","900,59360","900,74760","900,89360"
40,"1600,93280","1600,123280","1600,152480","1600,180880"


**Inutilisable: impossible d'avoir à la fois n_var et n_clauses dans [1000,5000]**

## GraphOrderingPrinciple

In [46]:
Lnodes = list(range(10,50,10))
Ledge = list(range(10,50,10))
Lres = [[] for _ in range(len(Lnodes))]
for i,n_nodes in enumerate(Lnodes):
    for edge in Ledge:
        G1 = nx.gnm_random_graph(n_nodes, edge,seed=random.randint(0,1000))
        F = GraphOrderingPrinciple(G1)
        res = str(stats(F,print_res=False))
        Lres[i].append(res)
    print('-',end='')
print()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display(pd.DataFrame(Lres, index=Lnodes, columns=Ledge))



----


,10,20,30,40
10,"90,775","90,775","90,775","90,775"
20,"380,7050","380,7050","380,7050","380,7050"
30,"870,24825","870,24825","870,24825","870,24825"
40,"1560,60100","1560,60100","1560,60100","1560,60100"


In [47]:
Lnodes = list(range(10,50,10))
Ledge = list(range(10,50,10))
Lres = [[] for _ in range(len(Lnodes))]
for i,n_nodes in enumerate(Lnodes):
    for edge in Ledge:
        G1 = nx.gnm_random_graph(n_nodes, edge,seed=random.randint(0,1000))
        F = GraphOrderingPrinciple(G1,plant=True)
        res = str(stats(F,print_res=False))
        Lres[i].append(res)
    print('-',end='')
print()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display(pd.DataFrame(Lres, index=Lnodes, columns=Ledge))

----


,10,20,30,40
10,"90,774","90,774","90,774","90,774"
20,"380,7049","380,7049","380,7049","380,7049"
30,"870,24824","870,24824","870,24824","870,24824"
40,"1560,60099","1560,60099","1560,60099","1560,60099"


**Inutilisable: impossible d'avoir à la fois n_var et n_clauses dans [1000,5000]**

## OrderingPrinciple

In [52]:
Lsizes = list(range(10,61,10))
for size in Lsizes:
    F = OrderingPrinciple(size,plant=True)
    res = str(stats(F,print_res=True))
print()

n_vars: 90, n_clauses: 774
n_vars: 380, n_clauses: 7049
n_vars: 870, n_clauses: 24824
n_vars: 1560, n_clauses: 60099
n_vars: 2450, n_clauses: 118874
n_vars: 3540, n_clauses: 207149



**Inutilisable: impossible d'avoir à la fois n_var et n_clauses dans [1000,5000]**

## PebblingFormula

In [58]:
Lnodes = list(range(1000,5001,1000))
for i,n_nodes in enumerate(Lnodes):
    G = nx.gnp_random_graph(n_nodes, p=0.5,directed = True,seed=random.randint(0,1000))
    DAG = nx.DiGraph([(u,v,{'weight':random.randint(-10,10)}) for (u,v) in G.edges() if u<v])
    F = PebblingFormula(DAG)
    res = str(stats(F,print_res=True))

n_vars: 1000, n_clauses: 1003
n_vars: 2000, n_clauses: 2003
n_vars: 3000, n_clauses: 3001
n_vars: 4000, n_clauses: 4002
n_vars: 5000, n_clauses: 5002


**Possible à utiliser mais long à générer (nécessité de faire varier les initialisations plutôt)**

## SparseStoneFormula

In [66]:
from networkx.algorithms import bipartite
Lnodes = list(range(10,51,10))
Lstones = list(range(2,20,3))
for i,n_nodes in enumerate(Lnodes):
    for n_stones in (Lstones):
        G = nx.gnp_random_graph(n_nodes, p=0.5,directed = True,seed=random.randint(0,1000))
        DAG = nx.DiGraph([(u,v,{'weight':random.randint(-10,10)}) for (u,v) in G.edges() if u<v])
        B = bipartite.random_graph(n_nodes, n_stones, 0.5, seed=random.randint(0,1000))
        F = SparseStoneFormula(DAG,B)
        res = str(stats(F,print_res=True))
    print('-'*10)

n_vars: 10, n_clauses: 13
n_vars: 29, n_clauses: 66
n_vars: 52, n_clauses: 52671
n_vars: 67, n_clauses: 45405
n_vars: 103, n_clauses: 487400
n_vars: 104, n_clauses: 2155048
----------
n_vars: 26, n_clauses: 28
n_vars: 63, n_clauses: 186530
n_vars: 94, n_clauses: 3216758


KeyboardInterrupt: 

**Difficilement utilisable: variations importantes du nombre de clauses**

## StoneFormula

In [67]:
Lnodes = list(range(10,51,10))
Lstones = list(range(2,20,3))
for i,n_nodes in enumerate(Lnodes):
    for n_stones in (Lstones):
        G = nx.gnp_random_graph(n_nodes, p=0.5,directed = True,seed=random.randint(0,1000))
        DAG = nx.DiGraph([(u,v,{'weight':random.randint(-10,10)}) for (u,v) in G.edges() if u<v])
        F = StoneFormula(DAG,n_stones)
        res = str(stats(F,print_res=True))
    print('-'*10)

n_vars: 22, n_clauses: 32
n_vars: 55, n_clauses: 7570
n_vars: 88, n_clauses: 1232978
n_vars: 121, n_clauses: 1234397


KeyboardInterrupt: 

**Generation trop longue et trop de variations**

## BinaryPigeonholePrinciple

In [77]:
Lpigeons = list(range(50,501,50))
Lholes = list(range(2,11,3))
for i,n_pigeons in enumerate(Lpigeons):
    for n_holes in (Lholes):
        F = BinaryPigeonholePrinciple(n_pigeons,n_holes)
        res = str(stats(F,print_res=True))
    print('-'*10)

n_vars: 50, n_clauses: 2450
n_vars: 150, n_clauses: 6275
n_vars: 150, n_clauses: 9800
----------
n_vars: 100, n_clauses: 9900
n_vars: 300, n_clauses: 25050
n_vars: 300, n_clauses: 39600
----------
n_vars: 150, n_clauses: 22350
n_vars: 450, n_clauses: 56325
n_vars: 450, n_clauses: 89400
----------
n_vars: 200, n_clauses: 39800
n_vars: 600, n_clauses: 100100
n_vars: 600, n_clauses: 159200
----------
n_vars: 250, n_clauses: 62250
n_vars: 750, n_clauses: 156375
n_vars: 750, n_clauses: 249000
----------
n_vars: 300, n_clauses: 89700
n_vars: 900, n_clauses: 225150
n_vars: 900, n_clauses: 358800
----------
n_vars: 350, n_clauses: 122150
n_vars: 1050, n_clauses: 306425
n_vars: 1050, n_clauses: 488600
----------
n_vars: 400, n_clauses: 159600
n_vars: 1200, n_clauses: 400200
n_vars: 1200, n_clauses: 638400
----------
n_vars: 450, n_clauses: 202050
n_vars: 1350, n_clauses: 506475
n_vars: 1350, n_clauses: 808200
----------
n_vars: 500, n_clauses: 249500
n_vars: 1500, n_clauses: 625250
n_vars: 1500

**Semble impossible d'obtenir nombre dev ariables et de clauses entre [1000,5000]**

## PigeonholePrinciple

In [78]:
Lpigeons = list(range(50,501,50))
Lholes = list(range(2,11,3))
for i,n_pigeons in enumerate(Lpigeons):
    for n_holes in (Lholes):
        F = PigeonholePrinciple(n_pigeons,n_holes)
        res = str(stats(F,print_res=True))
    print('-'*10)

n_vars: 100, n_clauses: 2500
n_vars: 250, n_clauses: 6175
n_vars: 400, n_clauses: 9850
----------
n_vars: 200, n_clauses: 10000
n_vars: 500, n_clauses: 24850
n_vars: 800, n_clauses: 39700
----------
n_vars: 300, n_clauses: 22500
n_vars: 750, n_clauses: 56025
n_vars: 1200, n_clauses: 89550
----------
n_vars: 400, n_clauses: 40000
n_vars: 1000, n_clauses: 99700
n_vars: 1600, n_clauses: 159400
----------
n_vars: 500, n_clauses: 62500
n_vars: 1250, n_clauses: 155875
n_vars: 2000, n_clauses: 249250
----------
n_vars: 600, n_clauses: 90000
n_vars: 1500, n_clauses: 224550
n_vars: 2400, n_clauses: 359100
----------
n_vars: 700, n_clauses: 122500
n_vars: 1750, n_clauses: 305725
n_vars: 2800, n_clauses: 488950
----------
n_vars: 800, n_clauses: 160000
n_vars: 2000, n_clauses: 399400
n_vars: 3200, n_clauses: 638800
----------
n_vars: 900, n_clauses: 202500
n_vars: 2250, n_clauses: 505575


KeyboardInterrupt: 

**Explosion du nombre de clauses par rapport au nombre de variables, semble impossible d'obtenir un nombre de clauses et variables dans [1000,5000]**

## PitfallFormula

In [81]:
Lv = range(6, 100, 10)
Ld = range(6, 100, 10)
Lny = range(6, 100, 10)
Lnz = range(6, 100, 10)
Lk =  range(2, 100, 10)
for v,d,ny,nz,k in (list(product(Lv,Ld,Lny,Lnz,Lk))):
    F = PitfallFormula(v,d,ny,nz,k)
    print(f"with v={v}, d={d}, ny={ny}, nz={nz}, k={k}, ",end="")
    res = str(stats(F,print_res=True))
    

ValueError: No regular 6-degree graph with 6-vertices exists.
Degree d must less than the number v of vertices,
and d*v must be even.

## PythagoreanTriples

In [1]:
Lk = list(range(50,501,50))
Ln = list(range(50,501,50))
Lm = list(range(50,501,50))
for k,n,m in (list(product(Lk,Ln,Lm))):
    F = RandomKCNF(random.randint(1,100),n,m)
    res = str(stats(F,print_res=True))
    

NameError: name 'PythagoreanTriples' is not defined

## KCNF

Generation possible mais trop simples.
Augmenter la taille des clauses n'est pas une bonne idée car l'assigment les rend sat tout le temps après.
(avec 100 var par clauses Pr(non-sat, clause_i) = (1-0.5)^100)

In [6]:
Lnodes = list(range(100,501,100))
Ledges = list(range(100,501,100))
clique_size = list(range(10,51,10))
for i,n_nodes in enumerate(Lnodes):
    for n_edges in (Ledges):
        for c in clique_size:
            G = nx.gnm_random_graph(n_nodes, n_edges,seed=random.randint(0,1000))
            F = BinaryCliqueFormula(G,c)
            res = str(stats(F,print_res=True))
        


n_vars: 70, n_clauses: 445780


KeyboardInterrupt: 

In [8]:
Lnodes = list(range(100,501,100))
colorings = list(range(2,10,1))
clique_size = list(range(2,51,10))
for i,n_nodes in enumerate(Lnodes):
    for c in (clique_size):
        for co in (colorings):
            F = CliqueColoring(n_nodes, c, co)
            res = str(stats(F,print_res=True))
        


n_vars: 5350, n_clauses: 30002
n_vars: 5450, n_clauses: 35152
n_vars: 5550, n_clauses: 40402
n_vars: 5650, n_clauses: 45752
n_vars: 5750, n_clauses: 51202
n_vars: 5850, n_clauses: 56752
n_vars: 5950, n_clauses: 62402
n_vars: 6050, n_clauses: 68152
n_vars: 6350, n_clauses: 729512


KeyboardInterrupt: 

In [11]:
import time
Lnodes = list(range(500,1001,100))
Ledges = list(range(100,501,100))
for i,n_nodes in enumerate(Lnodes):
    for n_edges in (Ledges):
            G = nx.gnm_random_graph(n_nodes, n_edges,seed=int(time.time()))
            F = TseitinFormula(G)
            res = str(stats(F,print_res=True))
        


n_vars: 100, n_clauses: 207
n_vars: 200, n_clauses: 461
n_vars: 300, n_clauses: 796
n_vars: 400, n_clauses: 1277
n_vars: 500, n_clauses: 1886
n_vars: 100, n_clauses: 203
n_vars: 200, n_clauses: 424
n_vars: 300, n_clauses: 698
n_vars: 400, n_clauses: 1011
n_vars: 500, n_clauses: 1529
n_vars: 100, n_clauses: 203
n_vars: 200, n_clauses: 420
n_vars: 300, n_clauses: 677
n_vars: 400, n_clauses: 957
n_vars: 500, n_clauses: 1344
n_vars: 100, n_clauses: 202
n_vars: 200, n_clauses: 416
n_vars: 300, n_clauses: 658
n_vars: 400, n_clauses: 935
n_vars: 500, n_clauses: 1259
n_vars: 100, n_clauses: 202
n_vars: 200, n_clauses: 415
n_vars: 300, n_clauses: 648
n_vars: 400, n_clauses: 904
n_vars: 500, n_clauses: 1197
n_vars: 100, n_clauses: 201
n_vars: 200, n_clauses: 406
n_vars: 300, n_clauses: 639
n_vars: 400, n_clauses: 895
n_vars: 500, n_clauses: 1165


In [13]:
import time
import numpy as np
Ln_levels = list(range(10,51,10))
Ln_nodes_per_level = 2**np.array(list(range(1,6,1)))
Ln_colors = 2**np.array(list(range(1,6,1)))
for i,n_levels in enumerate(Ln_levels):
    for n_nodes_per_level in (Ln_nodes_per_level):
        for n_colors in (Ln_colors):
            F = CPLSFormula(n_levels, n_nodes_per_level, n_colors)
            res = str(stats(F,print_res=True))
        



n_vars: 62, n_clauses: 78
n_vars: 104, n_clauses: 156
n_vars: 186, n_clauses: 312
n_vars: 348, n_clauses: 624
n_vars: 670, n_clauses: 1248
n_vars: 164, n_clauses: 298
n_vars: 248, n_clauses: 596
n_vars: 412, n_clauses: 1192
n_vars: 736, n_clauses: 2384
n_vars: 1380, n_clauses: 4768
n_vars: 408, n_clauses: 1170
n_vars: 576, n_clauses: 2340
n_vars: 904, n_clauses: 4680
n_vars: 1552, n_clauses: 9360
n_vars: 2840, n_clauses: 18720
n_vars: 976, n_clauses: 4642
n_vars: 1312, n_clauses: 9284
n_vars: 1968, n_clauses: 18568
n_vars: 3264, n_clauses: 37136
n_vars: 5840, n_clauses: 74272
n_vars: 2272, n_clauses: 18498
n_vars: 2944, n_clauses: 36996
n_vars: 4256, n_clauses: 73992
n_vars: 6848, n_clauses: 147984
n_vars: 12000, n_clauses: 295968
n_vars: 122, n_clauses: 158
n_vars: 204, n_clauses: 316
n_vars: 366, n_clauses: 632
n_vars: 688, n_clauses: 1264
n_vars: 1330, n_clauses: 2528
n_vars: 324, n_clauses: 618
n_vars: 488, n_clauses: 1236
n_vars: 812, n_clauses: 2472
n_vars: 1456, n_clauses: 4944
